# A 2D grain boundary in Boron Nitride

## 0. Introduction

This notebook demonstrates how to generate a 2D grain boundary in Boron Nitride, following the example in the manuscript:

> **Qiucheng Li, Xiaolong Zou, Mengxi Liu, Jingyu Sun, Yabo Gao, Yue Qi, Xiebo Zhou, Boris I. Yakobson, Yanfeng Zhang, and Zhongfan Liu**
> "Grain Boundary Structures and Electronic Properties of Hexagonal Boron Nitride on Cu(111)"
> *ACS Nano* **2015** 9 (6), 6308-6315
> [DOI: 10.1021/acs.nanolett.5b01852](https://doi.org/10.1021/acs.nanolett.5b01852)

Reproducing the material from Figure 2. c:

<img src="https://github.com/Exabyte-io/documentation/raw/12617167278ae3523adc028583b21ea4e8ebd197/images/tutorials/materials/defects/defect_planar_grain_boundary_2d_boron_nitride/0-figure-from-manuscript.webp" alt="Grain Boundary in Boron Nitride" width="400"/>

## 1. Prepare the Environment
### 1.1. Set up the notebook
Set the following flags to control the notebook behavior.


In [9]:
# Material selection
MATERIAL_NAME = "Boron_Nitride"  # Name of the material to import from Standata

# Grain boundary parameters
TARGET_TWIST_ANGLE = 9.0  # in degrees
BOUNDARY_GAP = 0.0  # Gap between two orientations in X direction, in Angstroms
OVERLAP_TOLERANCE = 0.8  # Tolerance to remove overlapping atoms, in Angstroms

# Search algorithm parameters
MAX_REPETITION = None  # Maximum supercell matrix element value
ANGLE_TOLERANCE = 0.5  # in degrees
RETURN_FIRST_MATCH = True  # If True, returns first solution within tolerance

# Visualization parameters
SHOW_INTERMEDIATE_STEPS = True
CELL_REPETITIONS_FOR_VISUALIZATION = [3, 3, 1]

### 1.2. Install packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install`.

In [10]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install("https://exabyte-io.github.io/api-examples/mat3ra_api_examples-0.1.dev1+gc41b0b8-py3-none-any.whl", deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages

    await install_packages("specific_examples")

### 1.3. Load and preview input material

In [11]:
from mat3ra.standata.materials import Materials
from mat3ra.made.material import Material

material = Material.create(Materials.get_by_name_first_match(MATERIAL_NAME))

## 2. Prepare Material
### 2.1. Select and visualize initial material

In [12]:
from utils.visualize import visualize_materials

if SHOW_INTERMEDIATE_STEPS:
    visualize_materials(material, repetitions=CELL_REPETITIONS_FOR_VISUALIZATION)

GridBox(children=(VBox(children=(Label(value='BN - Material - rotation: 0x,0y,0z', layout=Layout(align_self='c…

## 3. Generate Surface Grain Boundary
### 3.1. Set up grain boundary configuration and builder


In [13]:
from mat3ra.esse.models.core.reusable.axis_enum import AxisEnum
from mat3ra.made.tools.build.grain_boundary import create_grain_boundary_linear

grain_boundary = create_grain_boundary_linear(
    material=material,
    target_angle=TARGET_TWIST_ANGLE,
    angle_tolerance=ANGLE_TOLERANCE,
    max_repetition_int=7,
    return_first_match=RETURN_FIRST_MATCH,
    direction=AxisEnum.x,
    gap=BOUNDARY_GAP,
)

# We need to remove overlapping atoms at the boundary line
grain_boundary.basis.resolve_colliding_coordinates(tolerance=OVERLAP_TOLERANCE)

### 3.2. Generate and analyze grain boundaries


In [14]:
actual_angle = grain_boundary.metadata.build[-1].configuration.get("actual_angle", "unknown")
print(f"Target angle: {TARGET_TWIST_ANGLE}°")
print(f"Actual angle: {actual_angle}°")
print(f"Number of atoms: {len(grain_boundary.basis.elements.ids)}")

Target angle: 9.0°
Actual angle: 9.43°
Number of atoms: 146


## 4. Preview the grain boundary

In [15]:
visualize_materials(grain_boundary, title="Grain Boundary", viewer="wave")

<IPython.core.display.Javascript object>

### 5. Pass data to the outside runtime


In [16]:
from utils.jupyterlite import download_content_to_file, set_materials

set_materials(grain_boundary)
download_content_to_file(grain_boundary, "grain_boundary_2d_boron_nitride.json")

Data for materials written to uploads/BN(001)-BN(001), Linear Grain Boundary, 9.43 degrees.json


<IPython.core.display.Javascript object>